In [0]:
from pyspark.sql.functions import *

Ucitavanje prvog dataseta sa informacijama o igracima

In [0]:
file1_path = "/tmp/PlayersDF"

playersDF = spark.read.csv(file1_path, header = True, inferSchema = True)
playersDF.createOrReplaceTempView("players_table")
playersDF.show(truncate = False)
playersDF.printSchema()

+-------------------+----------+--------+--------+------+------+-----------------+--------------------------------------+
|name               |year_start|year_end|position|height|weight|birth_date       |college                               |
+-------------------+----------+--------+--------+------+------+-----------------+--------------------------------------+
|Alaa Abdelnaby     |1991      |1995    |F-C     |6-10  |240   |June 24, 1968    |Duke University                       |
|Mahmoud Abdul-Rauf |1991      |2001    |G       |6-1   |162   |March 9, 1969    |Louisiana State University            |
|Tariq Abdul-Wahad  |1998      |2003    |F       |6-6   |223   |November 3, 1974 |San Jose State University             |
|Shareef Abdur-Rahim|1997      |2008    |F       |6-9   |225   |December 11, 1976|University of California              |
|Alex Abrines       |2017      |2018    |G-F     |6-6   |190   |August 1, 1993   |null                                  |
|Alex Acker         |200

In [0]:
playersDF = playersDF.withColumn("birth_date", col("birth_date").cast("date"))
playersDF.printSchema()

root
 |-- name: string (nullable = true)
 |-- year_start: integer (nullable = true)
 |-- year_end: integer (nullable = true)
 |-- position: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- college: string (nullable = true)



Prikazivanje ukupnog broja igraca po godini koji nisu pohadjali univerzitet pre dolaska u NBA, ili su internacionalni igraci koji su direktno iz prethodnih klubova dosli u NBA

Grafik prikazuje iznenadan uspon ovakvog broja igraca od 2001. godine, kada se okvirno popularizovalo draftovanje igraca pravo iz srednje skole, zbog naglog uspeha prethodno drafrovanih igraca na taj nacin (prvenstveno Kobe Bryant i Kevin Garnett), i postepenog pada od 2007. godine, kada je uvedeno "One And Done" pravilo, koje se odnosi na to da igraci moraju da zavrse minimum jednu godinu studiranja na univerzitetu, kako bi bili dostupni za draft. (zbog toga svi igraci nakon 2007. godine koji nisu isli na univerzitet su igraci koji su dosli iz nekih profesionalnih klubova, uglavnom internacionalni igraci)

In [0]:
%sql
SELECT year_start as year, count(*) as number_of_players
FROM players_table
WHERE college is null
GROUP BY year_start
ORDER BY year_start DESC

year,number_of_players
2018,11
2017,15
2016,15
2015,13
2014,11
2013,12
2012,6
2011,8
2010,6
2009,9


Prikaz prosecnog trajanja karijere igraca po pozicijama u NBA ligi od 1990. godine do 2021. godine

Kao sto je i ocekivano, s obzirom na veliki broj igraca koji se u jednom periodu smatrao aktivnim u NBA ligi a jedva su igrali, pa samim tim i nisu dugo potrajali u ligi, iako vecina NBA zvezda ima karijeru od 12-15 godina, prosecna duzina karijere NBA igraca je 4-5 godina.

In [0]:
%sql
SELECT position, round(avg(year_end - year_start),2) as avg_career_length
FROM players_table
GROUP BY position
ORDER BY position DESC

position,avg_career_length
G-F,5.29
G,4.2
F-G,8.17
F-C,6.57
F,4.22
C-F,4.76
C,4.36


Ucitavanje drugog dataseta koji se odnosi na statistiku igraca za svaku NBA sezonu od 1990. do 2021.

In [0]:
file2_path = "/tmp/season_statsDF/"

statsDF = spark.read.csv(file2_path, header = True, inferSchema = True)
statsDF.createOrReplaceTempView("stats_table")
statsDF.display()
statsDF.printSchema()

Player,Year,Age,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
Mark Acres,1990,27,80,1691,138,285.0,69.2,135,281,48.0,3,4,75.0,83,120,69.2,154,277,431,67,36,25,70,248,362,8.3
Michael Adams,1990,27,79,2690,398,989.0,85.0,240,557,43.1,158,432,36.6,267,314,85.0,49,176,225,495,121,3,141,133,1221,15.4
Mark Aguirre,1990,30,78,2005,438,898.0,75.6,407,805,50.6,31,93,33.3,192,254,75.6,117,188,305,145,34,19,121,201,1099,15.8
Danny Ainge,1990,30,75,2727,506,1154.0,83.1,398,865,46.0,108,289,37.4,222,267,83.1,69,257,326,453,113,18,185,238,1342,16.1
Mark Alarie,1990,26,82,1893,371,785.0,81.2,361,736,49.0,10,49,20.4,108,133,81.2,151,223,374,142,60,39,101,219,860,14.1
Steve Alford,1990,25,41,302,63,138.0,94.6,56,116,48.3,7,22,31.8,35,37,94.6,2,23,25,39,15,3,16,22,168,18.7
Randy Allen,1990,25,63,746,106,239.0,53.5,106,232,45.7,0,7,0.0,23,43,53.5,49,89,138,23,16,19,28,102,235,8.4
Greg Anderson,1990,25,60,1291,219,432.0,53.5,219,432,50.7,0,0,0.0,91,170,53.5,112,261,373,24,32,54,80,176,529,13.0
Nick Anderson,1990,22,81,1785,372,753.0,70.5,371,736,50.4,1,17,5.9,186,264,70.5,107,209,316,124,69,34,138,140,931,14.9
Richard Anderson,1990,29,54,604,88,211.0,78.3,51,111,45.9,37,100,37.0,18,23,78.3,33,94,127,55,20,9,26,64,231,13.3


root
 |-- Player: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- G: integer (nullable = true)
 |-- MP: integer (nullable = true)
 |-- FG: integer (nullable = true)
 |-- FGA: double (nullable = true)
 |-- FG%: double (nullable = true)
 |-- 2P: integer (nullable = true)
 |-- 2PA: integer (nullable = true)
 |-- 2P%: double (nullable = true)
 |-- 3P: integer (nullable = true)
 |-- 3PA: integer (nullable = true)
 |-- 3P%: double (nullable = true)
 |-- FT: integer (nullable = true)
 |-- FTA: integer (nullable = true)
 |-- FT%: double (nullable = true)
 |-- ORB: integer (nullable = true)
 |-- DRB: integer (nullable = true)
 |-- TRB: integer (nullable = true)
 |-- AST: integer (nullable = true)
 |-- STL: integer (nullable = true)
 |-- BLK: integer (nullable = true)
 |-- TOV: integer (nullable = true)
 |-- PF: integer (nullable = true)
 |-- PTS: integer (nullable = true)
 |-- PER: double (nullable = true)



Prikaz evolucije suta za 3 poena tokom godina, i kako se sama igra drasticno promenila za okvirno mali vremenski period.

Nagli porast koji je prikazan na grafiku, od 1995-1997 godine, je opravdan time sto je NBA liga odlucila da u te tri sezone (1994-95, 1995-96, 1996-97) priblizi liniju za tri poena kosu, sa 7,25m na 6,70m, pa su igraci sa razlogom pokusavali vise suteva za 3 poena nego ranije.

In [0]:
%sql
SELECT Year, sum(3PA) as 3PT_attempts
FROM stats_table
where 3PA > 0
GROUP BY year
ORDER BY year ASC

Year,3PT_attempts
1990,15361
1991,16587
1992,17433
1993,20384
1994,23031
1995,35874
1996,41535
1997,44011
1998,34026
1999,20261


Prikaz 10 najboljih individualnih sezona od 1990. do 2021. gledajuci parametar PER (Player Efficiency Rating).

PER je parametar koji , kroz jednu veliku formulu, pokupi sve parametre igraca koji uticu na njegov performans tokom sezone, i pokusava da prikaze njegov doprinos tokom sezone u jednom broju. Radi poredjenja, prosecan PER svih igraca po sezoni iznosi 15.

Zanimljivo je da, od ovih 10 sezona, 9 su nagradjene MVP priznanjem.

In [0]:
%sql
SELECT Player, Age, Year, round(PTS/G,2) as PPG, round(TRB/G ,2) as RPG, round(AST/G ,2) as APG, round(STL/G ,2) as SPG, round(BLK/G ,2) as BPG, PER
FROM stats_table
where G > 50
ORDER BY PER DESC LIMIT 10

Player,Age,Year,PPG,RPG,APG,SPG,BPG,PER
LeBron James,24,2009,28.44,7.57,7.25,1.69,1.15,31.7
LeBron James,28,2013,26.79,8.03,7.25,1.7,0.88,31.6
Michael Jordan*,27,1991,31.46,6.0,5.52,2.72,1.01,31.6
Stephen Curry,27,2016,30.06,5.44,6.67,2.14,0.19,31.5
Michael Jordan*,26,1990,33.57,6.89,6.33,2.77,0.66,31.2
LeBron James,25,2010,29.71,7.29,8.57,1.64,1.01,31.1
Anthony Davis,21,2015,24.35,10.24,2.19,1.47,2.94,30.8
LeBron James,27,2012,27.15,7.94,6.24,1.85,0.81,30.7
David Robinson*,28,1994,29.79,10.69,4.76,1.74,3.31,30.7
Russell Westbrook,28,2017,31.58,10.67,10.37,1.64,0.38,30.6


### Prva hipoteza: 

Igrac je u najboljim godinama u NBA kada ima 27-28 godina.

Uzimamo u obzir igrace koji su bili aktivni u ligi 10 godina ili vise da bi imali dovoljno sezona za razmatranje

In [0]:
eligible_players = playersDF.select("*").where(col("year_end") - col("year_start") >= 10).drop("birth_date")
eligible_players.display()

name,year_start,year_end,position,height,weight,college
Mahmoud Abdul-Rauf,1991,2001,G,6-1,162,Louisiana State University
Shareef Abdur-Rahim,1997,2008,F,6-9,225,University of California
Arron Afflalo,2008,2018,G,6-5,210,"University of California, Los Angeles"
LaMarcus Aldridge,2007,2018,F-C,6-11,260,University of Texas at Austin
Victor Alexander,1992,2002,C-F,6-9,265,Iowa State University
Ray Allen,1997,2014,G,6-5,205,University of Connecticut
Tony Allen,2005,2018,G-F,6-4,213,Oklahoma State University
Rafer Alston,2000,2010,G,6-2,171,"California State University, Fresno"
Chris Andersen,2002,2017,F-C,6-10,245,Blinn College
Alan Anderson,2006,2017,G-F,6-6,220,Michigan State University


Spajamo tabelu igraca koji ulaze u opseg sa njihovim statistikama po sezonama

In [0]:
players_seasons = eligible_players.join(statsDF, eligible_players.name == statsDF.Player, "inner")
players_seasons = players_seasons.drop("year_start","year_end","position","height","weight","college","name")
players_seasons.createOrReplaceTempView("ps_table")
players_seasons.display()

Player,Year,Age,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
Nick Anderson,1990,22,81,1785,372,753.0,70.5,371,736,50.4,1,17,5.9,186,264,70.5,107,209,316,124,69,34,138,140,931,14.9
B.J. Armstrong,1990,22,81,1291,190,392.0,88.5,187,386,48.4,3,6,50.0,69,78,88.5,19,83,102,199,46,6,83,105,452,11.9
Dana Barros,1990,22,81,1630,299,738.0,80.9,204,500,40.8,95,238,39.9,89,110,80.9,35,97,132,205,53,1,123,97,782,12.8
Mookie Blaylock,1990,22,50,1267,212,571.0,77.8,194,491,39.5,18,80,22.5,63,81,77.8,42,98,140,210,82,14,111,110,505,11.5
Chucky Brown,1990,21,75,1339,210,447.0,76.2,210,440,47.7,0,7,0.0,125,164,76.2,83,148,231,50,33,26,69,148,545,11.9
Terry Davis,1990,22,63,884,122,262.0,62.1,122,261,46.7,0,1,0.0,54,87,62.1,93,136,229,25,25,28,68,171,298,9.0
Vlade Divac,1990,21,82,1611,274,549.0,70.8,274,544,50.4,0,5,0.0,153,216,70.8,167,345,512,75,79,114,110,240,701,17.5
Sherman Douglas,1990,23,81,2470,463,938.0,68.7,458,907,50.5,5,31,16.1,224,326,68.7,70,136,206,619,145,10,246,187,1155,17.0
Mike Dunleavy,1990,35,5,43,4,14.0,87.5,2,5,40.0,2,9,22.2,7,8,87.5,0,2,2,10,1,0,8,7,17,5.1
Sean Elliott,1990,21,81,2032,311,647.0,86.6,310,638,48.6,1,9,11.1,187,216,86.6,127,170,297,154,45,14,112,172,810,12.5


Broj igraca na po broju godina u ligi tokom svih posmatranih sezona

In [0]:
display(players_seasons.groupBy("Age").count().orderBy("Age"))

Age,count
18,6
19,37
20,80
21,159
22,276
23,370
24,390
25,394
26,428
27,426


Prosecan broj godina igraca u tim sezonama

In [0]:
players_seasons.select(mean("Age").alias("MeanAge")).display()

MeanAge
28.03336355394379


Prikaz prosecne statistike koja se odnosi na poene, skokove i asistencije po utakmici za sve igrace u tom periodu, po broju godina

In [0]:
%sql
SELECT Age, round(sum(PTS/G)/count(Age),2) as PPG, round(sum(TRB/G)/count(Age),2) as RPG, round(sum(AST/G)/count(Age),2) as APG
FROM ps_table
where G > 50
GROUP BY AGE
ORDER BY AGE ASC

Age,PPG,RPG,APG
18,7.32,3.03,1.41
19,9.84,4.2,1.89
20,10.74,4.72,2.22
21,11.99,4.91,2.36
22,11.49,4.71,2.41
23,12.14,4.89,2.62
24,12.67,4.97,2.79
25,12.72,4.93,2.75
26,12.55,5.0,2.81
27,12.86,4.96,2.87


Prikaz broja ukupnog broja igraca po godinama koji su imali sezonu sa PER-om preko 15

In [0]:
%sql
SELECT Age, count(PER) as PER
FROM ps_table
where G > 50 AND PER > 15
GROUP BY AGE
ORDER BY AGE ASC

Age,PER
18,1
19,10
20,30
21,51
22,90
23,123
24,143
25,156
26,155
27,149


Svaka analiza odradjena za vazniju statistiku po broju godina, gledajuci igrace koji ulaze u opseg, govori da je ova hipoteza tacna jer pokazuje na to da, kada igrac ima relativno najbolje sezone karijere, nalazi se u opsegu od 24 godine (kada se kaze da tek ulazi u prime) do 29 godina (kada uglavnom pocinje pad), sa akcentom na tome da 27 i 28 godina prednjace u prakticno svim analizama.

###Druga hipoteza:
Evolucija kosarke je uticala na to da danas visoki igraci (krila i centri) mnogo vise sutiraju za 3 poena nego pre 30 godina.

Uzimamo u obzir samo igrace cija je pozicija krilni centar (F-C) ili centar (C)

In [0]:
tall_players = playersDF.filter((playersDF.position.like('C%')) | (playersDF.position.like('F-C%')))
tall_players = tall_players.drop("year_start","year_end","weight","birth_date","college")
tall_players.display()

name,position,height
Alaa Abdelnaby,F-C,6-10
Steven Adams,C,7-0
Bam Adebayo,C-F,6-10
Alexis Ajinca,C,7-2
Solomon Alabi,C,7-1
Furkan Aldemir,F-C,6-10
Cole Aldrich,C,6-11
LaMarcus Aldridge,F-C,6-11
Victor Alexander,C-F,6-9
Jarrett Allen,F-C,6-10


Spajamo tabelu igraca sa tabelom statistike po sezonama i odbacujemo nepotrebne kolone za analizu

In [0]:
tp_seasons = tall_players.join(statsDF, eligible_players.name == statsDF.Player, "inner")
tp_seasons = tp_seasons.drop("name","Age","PTS","ORB","DRB","TRB","AST","STL","BLK","PER","PF","TOV")
tp_seasons.createOrReplaceTempView("tps_table")
tp_seasons.display()

position,height,Player,Year,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%
C,6-9,Torgeir Bryn,1990,3,10,0,2.0,66.7,0,2,0.0,0,0,0.0,4,6,66.7
F-C,6-8,Adrian Caldwell,1990,51,331,42,76.0,46.4,42,76,55.3,0,0,0.0,13,28,46.4
F-C,6-9,Terry Davis,1990,63,884,122,262.0,62.1,122,261,46.7,0,1,0.0,54,87,62.1
C,7-1,Vlade Divac,1990,82,1611,274,549.0,70.8,274,544,50.4,0,5,0.0,153,216,70.8
F-C,6-9,Pervis Ellison,1990,34,866,111,251.0,62.8,111,249,44.6,0,2,0.0,49,78,62.8
F-C,6-10,Shawn Kemp,1990,81,1120,203,424.0,73.6,201,412,48.8,2,12,16.7,117,159,73.6
F-C,6-11,Stacey King,1990,82,1777,267,530.0,72.7,267,529,50.5,0,1,0.0,194,267,72.7
C,7-1,Gary Leonard,1990,22,127,13,31.0,42.9,13,30,43.3,0,1,0.0,6,14,42.9
F-C,6-10,Clifford Robinson,1990,82,1565,298,751.0,55.0,286,707,40.5,12,44,27.3,138,251,55.0
C,6-11,Doug Roth,1990,42,412,37,86.0,50.0,37,85,43.5,0,1,0.0,7,14,50.0


U vizualizaciji je vec prikazana evolucija suta za 3 poena tokom godina, gde se na grafiku vidi porast u broju uzetih suteva za 3 poena tokom godina.
Sada treba analizirati da li je to slucaj i sa "velikim" igracima.

In [0]:
%sql
SELECT Year, sum(2P) - sum(3P) as 2Pvs3P, sum(2PA)-sum(3PA) as 2PAvs3PA, round(sum(3PA)/ sum(FGA),2) as PCT_of_shots_that_are_3P
FROM tps_table
GROUP BY Year
ORDER BY Year ASC

Year,2Pvs3P,2PAvs3PA,PCT_of_shots_that_are_3P
1990,1450,3044,0.03
1991,3148,6420,0.01
1992,4985,10034,0.02
1993,6945,13912,0.02
1994,9261,18537,0.01
1995,9449,18381,0.05
1996,12989,25774,0.04
1997,14169,28658,0.04
1998,15283,31218,0.02
1999,8627,17985,0.02


In [0]:
%sql
SELECT Year, sum(3P) as 3P, sum(3PA) as 3PA
FROM tps_table
GROUP BY Year
ORDER BY Year ASC

Year,3P,3PA
1990,27,101
1991,25,98
1992,46,178
1993,63,293
1994,54,276
1995,292,954
1996,330,1044
1997,352,1127
1998,203,741
1999,141,456


Vidimo da je u prethodnih par godina doslo do znatnog povecanja u broju pokusaja za 3 poena, kao i broju pogodaka. Posebno, u poslednjoj godini je doslo do drasticnog povecanja, tako da mozemo pretpostaviti da dolazi do eksponencijalnog rasta u sutiranju trojki kod visokih igraca. Samim tim, ovu hipotezu mogu potvrditi.

### Treca hipoteza:

### Structured streaming:

In [0]:
file_path = "/tmp/season_statsDF"

loadDF = spark.read.csv(file_path, header=True, inferSchema = True)
loadDF.show(10)
loadDF.printSchema()

+----------------+----+---+---+----+---+------+----+---+---+----+---+---+----+---+---+----+---+---+---+---+---+---+---+---+----+----+
|          Player|Year|Age|  G|  MP| FG|   FGA| FG%| 2P|2PA| 2P%| 3P|3PA| 3P%| FT|FTA| FT%|ORB|DRB|TRB|AST|STL|BLK|TOV| PF| PTS| PER|
+----------------+----+---+---+----+---+------+----+---+---+----+---+---+----+---+---+----+---+---+---+---+---+---+---+---+----+----+
|      Mark Acres|1990| 27| 80|1691|138| 285.0|69.2|135|281|48.0|  3|  4|75.0| 83|120|69.2|154|277|431| 67| 36| 25| 70|248| 362| 8.3|
|   Michael Adams|1990| 27| 79|2690|398| 989.0|85.0|240|557|43.1|158|432|36.6|267|314|85.0| 49|176|225|495|121|  3|141|133|1221|15.4|
|    Mark Aguirre|1990| 30| 78|2005|438| 898.0|75.6|407|805|50.6| 31| 93|33.3|192|254|75.6|117|188|305|145| 34| 19|121|201|1099|15.8|
|     Danny Ainge|1990| 30| 75|2727|506|1154.0|83.1|398|865|46.0|108|289|37.4|222|267|83.1| 69|257|326|453|113| 18|185|238|1342|16.1|
|     Mark Alarie|1990| 26| 82|1893|371| 785.0|81.2|361|736|49

In [0]:
schema = loadDF.schema

In [0]:
streamingDF = (
    spark
    .readStream
    .schema(schema)
    .option("maxFilesPerTrigger",1)
    .option("header","true")
    .csv(file_path)
)

streamingDF = (
    streamingDF
    .filter((col("PER") > 25) & (col("G") > 50))
    .groupBy(col("Year"))
    .count()
)
print(streamingDF.isStreaming)

True


In [0]:
query = (
    streamingDF
    .writeStream
    .format("memory")
    .queryName("counts")
    .outputMode("complete")
    .start()
)

In [0]:
%sql
SELECT * FROM counts 
ORDER BY Year ASC

Year,count
1990,6
1991,4
1992,3
1993,4
1994,3
1995,5
1996,6
1997,4
1998,4
2000,3


In [0]:
%sql
SELECT * FROM counts 
ORDER BY Year ASC

Year,count
1990,6
1991,4
1992,3
1993,4
1994,3
1995,5
1996,6
1997,4
1998,4
2000,3


In [0]:
%sql
SELECT * FROM counts 
ORDER BY Year ASC

Year,count
1990,6
1991,4
1992,3
1993,4
1994,3
1995,5
1996,6
1997,4
1998,4
2000,3


In [0]:
%sql
SELECT * FROM counts 
ORDER BY Year ASC

Year,count
1990,6
1991,4
1992,3
1993,4
1994,3
1995,5
1996,6
1997,4
1998,4
2000,3


In [0]:
%sql
SELECT * FROM counts 
ORDER BY Year ASC

Year,count
1990,6
1991,4
1992,3
1993,4
1994,3
1995,5
1996,6
1997,4
1998,4
2000,3


In [0]:
query.stop()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-4006617544408452> in <module>
----> 1 query.stop()

/databricks/spark/python/pyspark/sql/streaming.py in stop(self)
    155         """Stop this streaming query.
    156         """
--> 157         self._jsq.stop()
    158 
    159     def explain(self, extended=False):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    115     def deco(*a, **kw):
    116         try:
--> 117             return f(*a, **kw)
    118         except py4j.protocol.Py4JJavaError as e:
    119             converted = convert_except